# 0. Install and Import Dependencies

In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
from skimage.metrics import structural_similarity as ssim

# 1. Load Model

In [3]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_thunder_tflite_float16_4.tflite')
interpreter.allocate_tensors()

# 2. Make Detections

In [4]:
img = cv2.imread('Virtual tryon data\\test\\image\\00017_00.jpg')  # Replace with your image path
# cv2.imshow('MoveNet Lightning', img)
orig_img = img.copy()
img = cv2.resize(img, (256, 256)) 

In [107]:
input_image = tf.convert_to_tensor(img, dtype=tf.uint8)
input_image = tf.expand_dims(input_image, axis=0)

In [108]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [109]:
interpreter.set_tensor(input_details[0]['index'], input_image)
interpreter.invoke()
keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])

In [110]:

EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [111]:
# Rendering 
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [112]:
# Draw keypoints and connections on the image
draw_keypoints(img, keypoints_with_scores, 0.4)
draw_connections(img, keypoints_with_scores, EDGES, 0.4)

In [5]:
# Display result
img = cv2.resize(img, (768, 1024))

cv2.imshow('MoveNet Lightning', img)
cv2.imshow('Original', orig_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [114]:
img.shape

(1024, 768, 3)

In [115]:
orig_img.shape

(1024, 768, 3)

In [6]:
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2GRAY)
sim, diff = ssim(gray_img, gray_orig_img, full=True, multichannel=True)
print(f'Similarity:{sim}')
print(f'Difference:{diff}')

Similarity:0.9454944260150561
Difference:[[1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 ...
 [0.99813115 0.99813115 0.99869554 ... 1.         1.         1.        ]
 [0.99869391 0.99869391 0.99899838 ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]]
